# Make up v1

Referencia: https://www.pyimagesearch.com/2017/04/17/real-time-facial-landmark-detection-opencv-python-dlib/

## Part 3

In [ ]:
# import the necessary packages
from imutils.video import VideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import time
import dlib
import cv2

args = {}
args["shape_predictor"] = "shape_predictor_68_face_landmarks.dat"
args["picamera"] = 0

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

# initialize the video stream and allow the cammera sensor to warmup
print("[INFO] camera sensor warming up...")
vs = VideoStream(usePiCamera=args["picamera"] > 0).start()
time.sleep(2.0)

In [ ]:
def visualize_facial_landmarks(image, shape, colors=None, alpha=0.8):
    # create two copies of the input image -- one for the
    # overlay and one for the final output image
    overlay = image.copy()
    output = image.copy()
    height,width,channels = image.shape
    # if the colors list is None, initialize it with a unique
    # color for each facial landmark region
    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
            (168, 100, 168), (158, 163, 32),
            (163, 38, 32), (180, 42, 220)]
    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
        # grab the (x, y)-coordinates associated with the
        # face landmark
        (j, k) = FACIAL_LANDMARKS_IDXS[name]
        pts = shape[j:k]

        # check if are supposed to draw the jawline
        if name == "right_eyebrow": # or name == "left_eyebrow" or name=="mouth":
            # since the jawline is a non-enclosed facial region,
            # just draw lines between the (x, y)-coordinates
            text = 'Step 1: Paint your right eyebrown like this'
            font = cv2.FONT_HERSHEY_SIMPLEX
            
            radius = 20
            cv2.putText(overlay, text, (2*radius+5, height-10), font, 0.5, (255, 255, 0), 2)
            cv2.circle(overlay,(radius,height-radius), radius, (0,0,255), -1)
            for l in range(1, len(pts)):
                ptA = tuple(pts[l - 1])
                ptB = tuple(pts[l])
                cv2.line(overlay, ptA, ptB, colors[i], 2)

        # otherwise, compute the convex hull of the facial
        # landmark coordinates points and display it
        #else:
        #    hull = cv2.convexHull(pts)
        #    cv2.drawContours(overlay, [hull], -1, colors[i], -1)
    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    return output

In [ ]:
def makeup_step(new_name,color, image, shape, fill_shape, step_number, text,alpha=0.8):
    overlay = image.copy()
    output = image.copy()
    radius = 20
    height,width,channels = image.shape
    for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
        (j, k) = FACIAL_LANDMARKS_IDXS[name]
        pts = shape[j:k]
        if name == new_name:
            if fill_shape:
                hull = cv2.convexHull(pts)
                cv2.drawContours(overlay, [hull], -1, color, -1)
            else:
                for l in range(1, len(pts)):
                    ptA = tuple(pts[l - 1])
                    ptB = tuple(pts[l])
                    cv2.line(overlay, ptA, ptB, color, 2)
    cv2.rectangle(overlay,(2*radius,height-30),(250,height),(0,0,0),-1)
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.circle(output,(radius,height-radius), radius, color, -1)
    cv2.putText(output, text, (2*radius+5, height-10), font, 0.5, (255, 255, 255), 1)
    # return the output image
    return output
    

In [ ]:
from collections import OrderedDict
STEPS = OrderedDict([
    (3,"mouth"),
    (1,"right_eyebrow"),
    (2,"left_eyebrow"),
    (6,"right_eye"),
    (5,"left_eye"),
    (4,"nose"),
    (7,"jaw")
])
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 35)),
    ("jaw", (0, 17))
])
default_color = (0,170,220)
COLORS = OrderedDict([
    ("mouth", (0, 200, 0)),
    ("right_eyebrow", default_color),
    ("left_eyebrow", default_color),
    ("right_eye", default_color),
    ("left_eye", default_color),
    ("nose", default_color),
    ("jaw", default_color)
])
TEXTS = OrderedDict([
    ("mouth", "L'Oreal Lipstick Mate N.68"),
    ("right_eyebrow", "L'Oreal Eyeliner N.51"),
    ("left_eyebrow", "L'Oreal Eyeliner N.51"),
    ("right_eye", "L'Oreal Eyeliner N.51"),
    ("left_eye", "L'Oreal Eyeliner N.51"),
    ("nose", "L'Oreal Paris Infalible"),
    ("jaw", "Powder")
])

In [ ]:
current_step = 2
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=600)
    frame=cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    output = frame.copy()
    rects = detector(gray, 0)
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        #output = visualize_facial_landmarks(frame, shape)
        output = makeup_step(STEPS[current_step],COLORS[STEPS[current_step]], frame, shape, True,current_step,TEXTS[STEPS[current_step]],0.4)
    cv2.imshow("Frame", output)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

In [ ]:
cv2.destroyAllWindows()
vs.stop()